In [157]:
import pandas as pd
from datetime import datetime

In [158]:
df1 = pd.read_csv("paypal.csv", encoding='latin-1')

In [159]:
paypal = pd.DataFrame(df1, columns=['Data', ' Nome', ' Líquido'])

In [160]:
df2 = pd.read_excel('mercadopago.xls', sheet_name='0')

In [161]:
mercado_pago = pd.DataFrame(df2, columns=['Data de pagamento', 'Tipo de operação','Valor'])

In [162]:
df3 = pd.read_csv("caixa.txt", delimiter=";")

In [163]:
conta = pd.DataFrame(df3, columns=['Data_Mov','Historico', 'Valor','Deb_Cred'])

In [164]:
def format_data_paypal(data):
    return datetime.strptime(data, "%d-%m-%Y").strftime("%d/%m/%Y")

In [165]:
def format_data_mercadopago(data):
    return datetime.strptime(data, "%d/%m/%Y %H:%M").strftime("%d/%m/%Y")

In [166]:
def format_data_caixa(data):
    return datetime.strptime(str(data),"%Y%m%d").strftime("%d/%m/%Y")

In [167]:
paypal['Data'] = paypal['Data'].apply(format_data_paypal)

In [168]:
mercado_pago['Data de pagamento'] = mercado_pago['Data de pagamento'].apply(format_data_mercadopago)

In [169]:
conta['Data_Mov'] = conta['Data_Mov'].apply(format_data_caixa)

In [170]:
def format_valor_paypal(data):
    return float(data.replace(".","").replace(",","."))

In [171]:
paypal['valor'] =  paypal[' Líquido'].apply(format_valor_paypal)
paypal.drop(' Líquido', axis=1, inplace=True)

In [172]:
def format_valor_caixa(linha):
    if linha['Deb_Cred'] == 'D' and linha['Valor'] is not 0:
        return linha['Valor'] * -1
    else:
        return linha['Valor']

In [173]:
conta['Valor'] = conta.apply(format_valor_caixa, axis=1)

In [174]:
conta.drop('Deb_Cred',axis=1, inplace=True)

In [175]:
paypal["origem"] = "paypal"
mercado_pago["origem"] = "mercado pago"
conta["origem"] = "caixa"

In [176]:
paypal['valor'].sum() + mercado_pago['Valor'].sum() + conta['Valor'].sum()

-60.030000000000086

In [177]:
paypal.rename(columns={' Nome': 'quem'}, inplace=True)
mercado_pago.rename(columns={'Data de pagamento': 'Data', 'Valor':'valor','Tipo de operação':'quem'}, inplace=True)
conta.rename(columns={'Data_Mov': 'Data', 'Valor':'valor','Historico':'quem'}, inplace=True)

In [178]:
frames = [paypal,mercado_pago,conta]

In [179]:
fechamento = pd.concat(frames)

In [180]:
def format_valor_export(data):
    return data.replace(".",",")

In [181]:
fechamento.valor = fechamento.valor.astype(str);
fechamento.valor = fechamento.valor.apply(format_valor_export)
fechamento = fechamento[['Data', 'quem', 'origem', 'valor']]


In [182]:
print(fechamento)

          Data                                   quem        origem    valor
0   26/09/2018               Juliano Genari de Araujo        paypal    37,48
1   26/09/2018                     Gabriel R Ferreira        paypal     47,0
2   25/09/2018                           Gilson Filho        paypal    18,44
3   25/09/2018         JOAO RODRIGUES SILVEIRA JUNIOR        paypal    94,61
4   24/09/2018                         Jeronimo Filho        paypal    94,61
5   23/09/2018                Tulio Monteiro Da Silva        paypal    13,68
6   22/09/2018                           Fabio Araujo        paypal     47,0
7   22/09/2018          Luiz Felipe Sousa Melo Vieira        paypal     9,87
8   22/09/2018                           Fabio Araujo        paypal     47,0
9   22/09/2018            Alessandro Rizzatto Pignata        paypal     8,92
10  22/09/2018                          TANIA MIRANDA        paypal     8,92
11  22/09/2018            Hamlet Pessoa Farias Junior        paypal     47,0

In [183]:
fechamento.to_csv('out.csv')